In [1]:
import pandas as pd
import numpy as np
import string

In [2]:
comments = pd.read_csv('../../data/attack_annotated_comments.tsv', sep = '\t', index_col = 0)
annotations = pd.read_csv('../../data/attack_annotations.tsv',  sep = '\t')

In [3]:
labels = annotations.groupby('rev_id')['attack'].mean() > 0.5
comments['attack'] = labels

In [16]:
comments['comment'] = comments['comment'].apply(lambda x: x.replace("NEWLINE_TOKEN", " "))
comments['comment'] = comments['comment'].apply(lambda x: x.replace("TAB_TOKEN", " "))
comments['comment'] = comments['comment'].apply(lambda x: x.lower())
comments['comment'] = comments['comment'].apply(lambda x: "".join(l for l in x if l not in string.punctuation))

In [18]:
# table 1 info
print("# Posts:", len(comments))
print("# Classes:", len(comments["attack"].unique()))
commentlength = comments["comment"].apply(lambda x: len(x.split()))
lengthlimit = commentlength.quantile(.95)
print("95% length:", lengthlimit)
print("max length:", commentlength.max())

# Posts: 115864
# Classes: 2
95% length: 224.0
max length: 2832


In [14]:

words = {word for c in comments["comment"].values for word in c.split() if "http" not in word
                                                                        and "www" not in word}
print(len(words))
# far too many words compared to their method - but no code is available to check it.

180407


185800


In [20]:
truncated = comments["comment"].apply(lambda x: " ".join(x.split()[:int(lengthlimit)]))
df = pd.concat([truncated, comments.iloc[:,1:]], axis=1)

In [62]:
swearwordsfile = "../../swear_words.txt"
with open(swearwordsfile, "r") as f:
        swearwords = {x.strip() for x in f.readlines()}

print("is 'fuckshit' a swearword?", "fuckshit" in swearwords)
print("but are swearwords in 'fuckshit'?", any([x in 'fuckshit' for x in swearwords]))

is 'fuckshit' a swearword? False
but are swearwords in 'fuckshit'? True


In [64]:
from textblob import TextBlob

def tokens(post):
    return TextBlob(post).words

def get_bad_word_count(post):
    x = tokens(post)
    count = 0
    bad_words = []
    for word in x:
        if word in swearwords:
            count+=1
            bad_words.append(word)
    return count, str(bad_words)

bwc, bwl = zip(*comments["comment"].map(get_bad_word_count))
has_bw = pd.Series(np.array(bwc) > 0, index=df.index)
has_swears = df["comment"].apply(lambda x: any([swear in x for swear in swearwords]))
swears = pd.concat([has_bw, has_swears], axis=1, keys=["original", "new"])

In [55]:
np.sum(np.array(bwc) > 0) / len(bwc)
print("Comparison value for swear share: ", has_swears.value_counts(normalize=True).round(2)[True])
# df["has_swears"] = np.array(bwc) > 0

Comparison value for swear share:  0.41


In [68]:
pb = df["attack"].value_counts(normalize=True)[True]
ps1 = swears["original"].value_counts(normalize=True)[True]
ps2 = swears["new"].value_counts(normalize=True)[True]
pa = df["logged_in"].value_counts(normalize=True)[False]
# print(pb, ps, pa)

pb_s1 = df["attack"][swears["original"]].value_counts(normalize=True)[True]
pb_s2 = df["attack"][swears["new"]].value_counts(normalize=True)[True]
ps1_b = swears["original"][df["attack"]].value_counts(normalize=True)[True]
ps2_b = swears["new"][df["attack"]].value_counts(normalize=True)[True]
pb_a = df["attack"][~df["logged_in"]].value_counts(normalize=True)[True]
pa_b = df["logged_in"][df["attack"]].value_counts(normalize=True)[False]
ps1_a = swears["original"][~df["logged_in"]].value_counts(normalize=True)[True]
ps2_a = swears["new"][~df["logged_in"]].value_counts(normalize=True)[True]
pb_as1 = df["attack"][~df["logged_in"] & swears["original"]].value_counts(normalize=True)[True]
pb_as2 = df["attack"][~df["logged_in"] & swears["new"]].value_counts(normalize=True)[True]

# print(pb_s, ps_b, pb_a, pa_b, ps_a, pb_as)

table2 = pd.DataFrame([[pb, ps1, pa, pb_s1, ps1_b, pb_a, pa_b, ps1_a, pb_as1],
                    [pb, ps2, pa, pb_s2, ps2_b, pb_a, pa_b, ps2_a, pb_as2]],
                   columns = ["P(B)", "P(S)", "P(A)", "P(B|S)", "P(S|B)", "P(B|A)", "P(A|B)", "P(S|A)", "P(B|(A&S))"],
                   index = ["original", "new"])
print(table2.T.round(3))

           original    new
P(B)          0.117  0.117
P(S)          0.165  0.407
P(A)          0.267  0.267
P(B|S)        0.496  0.235
P(S|B)        0.696  0.815
P(B|A)        0.247  0.247
P(A|B)        0.562  0.562
P(S|A)        0.270  0.481
P(B| A&S)     0.653  0.428


In [35]:
df

,comment,year,logged_in,ns,sample,split,attack,has_swears
rev_id,,,,,,,,
37675,this is not creative those are the dictionary ...,2002,False,article,random,train,False,True
44816,the term standard model is itself less npov th...,2002,False,article,random,train,False,True
49851,true or false the situation as of march was su...,2002,False,article,random,train,False,False
89320,next maybe you could work on being less condes...,2002,True,article,random,dev,False,True
93890,this page will need disambiguation,2002,True,article,random,train,False,False
...,...,...,...,...,...,...,...,...
699848324,these sources dont exactly exude a sense of im...,2016,True,article,blocked,train,False,True
699851288,the institute for historical review is a peerr...,2016,True,article,blocked,test,False,True
699857133,the way youre trying to describe it in this ar...,2016,True,article,blocked,train,False,True
